<h1> Lecture 33 

Data Science 8, Spring 2021 </h1>

<h3>
<b>
<ul>
<li>Residuals (Continued)</li><br>
    <li>Regression Inference</li>
</ul>
</b>
</h3>

In [ ]:
from datascience import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
plots.rcParams["patch.force_edgecolor"] = True

#The following allows porting images into a Markdown window
#Syntax: ![title](image_name.png)
from IPython.display import Image

In [ ]:
def standard_units(arr):
    return (arr - np.average(arr))/np.std(arr)

def correlation(t, x, y):
    x_standard = standard_units(t.column(x))
    y_standard = standard_units(t.column(y))
    return np.average(x_standard * y_standard)

def slope(t, x, y):
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def intercept(t, x, y):
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

def fitted_values(t, x, y):
    """Return an array of the regression estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

def residuals(t, x, y):
    predictions = fitted_values(t, x, y)
    return t.column(y) - predictions

In [ ]:
def plot_fitted(t, x, y):
    tbl = t.select(x, y)
    tbl.with_columns('Fitted Value', fitted_values(t, x, y)).scatter(0)

In [ ]:
def plot_residuals(t, x, y):
    tbl = t.with_columns(
        'Fitted', fitted_values(t, x, y),
        'Residual', residuals(t, x, y)
    )
    tbl.select(x, y, 'Fitted').scatter(0)
    tbl.scatter(x, 'Residual')

## A Measure of Clustering ##

In [ ]:
heights = Table.read_table('heights.csv')

heights = Table().with_columns(
    'MidParent', heights.column('midparentHeight'),
    'Child', heights.column('childHeight')
    )
plot_residuals(heights, 'MidParent', 'Child')

<h4> Look at the vertical standard deviation of the actual heights&mdash;that is, $\sigma_y$&mdash;of the $𝑦$ values of the scatter plot, the blue points) from the Heights data set.</h4>

In [ ]:
child_observed_sd = np.std(heights.column('Child'))
np.round(child_observed_sd,2)

<h4>The above is the standard deviation of an uninformed (blind) estimate.<br>

In this case, the residual (error) is $e=y-\widehat{y}=y-\mu_y$, so the variance $\sigma_e^2$ of the error is simply the variance $\sigma_y^2$ of the observed values.<br>

<u>Note</u>: The above corresponds to the case where we're not given the parental heights, but we're asked to make a guess as to what the child's height is&mdash;that is, provide a best <u>"blind"</u> guess.<br>
    
We'd then use the mean of the children's heights as our best guess.  In that case, our error (residual) standard deviation $\sigma_e$ is simply equal to $\sigma_y$, the standard deviation of the children's heights.</h4>

<h4>What about the standard deviation of the error if we make a prediction <i>knowing</i> the heights of the parents?<br>
    
<u>Answer</u>: In that case, the standard deviation is again the standard deviation of the errors (residuals), but this time the parents' heights tell us something useful that makes the error standard deviation $\sigma_e$ smaller than the observation standard deviation $\sigma_y$&mdash;that is, $\sigma_e < \sigma_y$.<br>  
    
If the parents' heights were uncorrelated with the children's heights, we'd gain no benefit, and the error standard deviation would simply be $\sigma_e=\sigma_y$.<br>

In short, we're guaranteed that $\sigma_e \leq \sigma_y$.</h4>

<h4>Look at the standard deviation of the fitted values (predictions)&mdash;that is, $\sigma_{\widehat{y}}$&mdash;the $y$ values of the yellow line, from the Heights data set.</h4>

In [ ]:
child_predictions_sd = np.std(fitted_values(heights, 'MidParent', 'Child'))
np.round(child_predictions_sd,2)

<h4>The above is the standard deviation of an informed estimate.</h4>

<h4>So far we've looked at <br>
<ul><li>the standard deviation of the actual (observed) values; and</li><br>
        
<li>the standard deviation of the fitted values; </li><br>
</ul>
We'll now examine how they relate.
</h4>

<h5>Ratio $$\displaystyle \frac{\sigma_{\widehat{y}}}{\sigma_y}=\frac{\textsf{SD of the predictions (fitted values)}}{\textsf{SD of the actual values}}$$</h5>

In [ ]:
np.round(child_predictions_sd / child_observed_sd,2)

<h5>The correlation coefficient $r$ of the MidParent Height (i.e., $x$) and Child Height (i.e., $y$) is the same as the ratio above:</h5>

In [ ]:
np.round(correlation(heights, 'MidParent', 'Child'),2)

<h4>This is <i>not</i> a coincidence.</h4>   

<h5>Test the same thing on the dugong data set:</h5>

In [ ]:
dugong = Table.read_table('dugong.csv')

In [ ]:
dugong_prediction_sd = np.std(fitted_values(dugong, 'Length', 'Age'))
dugong_observed_sd = np.std(dugong.column(1))

<h5>Ratio $\displaystyle \frac{\sigma_{\widehat{y}}}{\sigma_y}$ of the SD of the predictions (fitted values) and the SD of the actual values:</h5>

In [ ]:
np.round(dugong_prediction_sd / dugong_observed_sd,2)

<h5>The correlation coefficient $r$ of the Dugong Length (i.e., $x$) and Dugong Age (i.e., $y$) is the same as the ratio above:</h5>

In [ ]:
np.round(correlation(dugong, 'Length', 'Age'),2)

<h4>So far, it seems as though $$\frac{\sigma_{\widehat{y}}}{\sigma_y}=r.$$
    
The following case example shows that some caution is needed.</h4>

<h3>Acceleration and MPG relationship in a data set of hybrid cars.</h3>

In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid.show(5)

In [ ]:
plot_residuals(hybrid, 'acceleration', 'mpg')

<h4>The residuals scatter plot above has a wedge shape.  It's <u>not</u> a patternless blob. It has <i>heteroskasticity&mdash;the variability changes (in this case, reduced) as we go along the horizontal axis.</i> (This is outside the scope of this course).</h4>

<h4>The correlation coefficient between Acceleration and MPG is negative, as expected.</h4>

In [ ]:
np.round(correlation(hybrid, 'acceleration', 'mpg'),2)

<h5>Ratio $\displaystyle \frac{\sigma_{\widehat{y}}}{\sigma_y}$ of the SD of the predictions (fitted values) and the SD of the actual values:</h5>

In [ ]:
np.round(np.std(fitted_values(hybrid, 'acceleration', 'mpg'))
         /np.std(hybrid.column('mpg')),2)

<h4>In this case, the correlation coefficient is the <i>negative</i> of the ratio of the two SDs&mdash;that is,<br> 

$$\frac{\sigma_{\widehat{y}}}{\sigma_y}=-r.$$.<br>
    
Note that the ratio of the SDs can never be negative, because SD is never negative. This is why the right-hand side of the equation above cannot be $r$.</h4>

<h4>
We can now put the above cases together ...<br>

No matter what the shape of the scatter plot, the SD of the predictions (fitted values $\widehat{y}$) is a fraction of the SD of the observed values of $y$. The fraction is |r|.

$$
\frac{\textsf{SD of Fitted Values}}{\textsf{SD of }y} = \frac{\sigma_{\widehat{y}}}{\sigma_{y}} = |r|.$$

This is because it can be shown mathematically that $\sigma_{\widehat{y}}=|r|\, \sigma_y$.  That is,<br> 

$$\textsf{SD of Fitted Values} = |r|\cdot \textsf{SD of }y.$$
</h4>


SLIDE: SD of Fitted Values (Predictions)

<h3> SD of the Residuals </h3>
No matter what the shape of the scatter plot, the SD of the residuals is a fraction of the SD of the observed values of $y$. The fraction is  $\sqrt{1-r^2}$.

$$
\textsf{SD of residuals} ~=~ \sqrt{1 - r^2} \cdot \textsf{SD of }y
$$

In [ ]:
plot_fitted(heights, 'MidParent', 'Child')

In [ ]:
plot_fitted(heights, 'MidParent', 'Child')
ave_child = np.mean(heights.column('Child'))
plots.plot([64, 76], [ave_child, ave_child]);

<h3>Child Heights</h3>

<h4>Variance of the Child Heights (based on blind estimation w/o knowing MidParent Heights)</h4>

In [ ]:
np.std(heights.column('Child')) ** 2

<h4>$\sigma_{e}^2$: Variance of the Residuals:</h4>

In [ ]:
np.round(np.std(residuals(heights, 'MidParent', 'Child')) ** 2,2)

<h5>Let's add the predictions (fitted values) and the residuals to the heights table:</h5>

In [ ]:
heights = heights.with_columns(
    'Fitted Value', fitted_values(heights, 'MidParent', 'Child'),
    'Residual', residuals(heights, 'MidParent', 'Child')
)
heights

<h4>$\sigma_{\widehat{y}}^2$: Variance of the Fitted Values (Predictions):</h4>

In [ ]:
np.round(np.std(heights.column('Fitted Value')) ** 2,2)

<h4>Note that:<br><br> Variance of the Residuals + Variance of Fitted Values = Variance of the Observed Child Heights:<br>

$$\sigma_e^2+\sigma_{\widehat{y}}^2=\sigma_y^2.$$</h4>

In [ ]:
np.round(
    np.std(residuals(heights, 'MidParent', 'Child')) ** 2 
    + np.std(heights.column('Fitted Value')) ** 2
,2)

<h3>Dugongs</h3>

$\sigma_y^2$:

In [ ]:
np.round(np.std(dugong.column('Age')) ** 2,2)

$\sigma_{\widehat{y}}^2$:

In [ ]:
np.round(np.std(fitted_values(dugong, 'Length', 'Age')) ** 2,2)

$\sigma_e^2$:

In [ ]:
np.round(np.std(residuals(dugong, 'Length', 'Age')) ** 2,2)

Easy to verify that $\sigma_e^2+\sigma_{\widehat{y}}^2=\sigma_y^2$.

<h4>Correlation Coefficient of Child Heights:</h4>

In [ ]:
r = correlation(heights, 'MidParent', 'Child')
r

In [ ]:
np.sqrt(1 - r**2) * np.std(heights.column('Child'))

<h4>Compare with the SD of the Residuals:</h4>

In [ ]:
np.std(residuals(heights,'MidParent','Child'))

<h4>Now try on the Hybrid Car Data:</h4>

In [ ]:
r = correlation(hybrid, 'acceleration', 'mpg')
r

In [ ]:
np.sqrt(1 - r**2)*np.std(hybrid.column('mpg'))

<h4>Compare with the SD of the Residuals:</h4>

In [ ]:
np.std(residuals(hybrid, 'acceleration', 'mpg'))

## Regression Model ##

In [ ]:
def draw_and_compare(true_slope, true_int, sample_size):
    x = np.random.normal(50, 5, sample_size)
    xlims = np.array([np.min(x), np.max(x)])
    errors = np.random.normal(0, 6, sample_size)
    y = (true_slope * x + true_int) + errors
    sample = Table().with_columns('x', x, 'y', y)

    sample.scatter('x', 'y')
    #The following line plots the true line in green
    plots.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plots.title('True Line, and Points Created')

    sample.scatter('x', 'y')
    plots.title('What We Get to See')

    sample.scatter('x', 'y', fit_line=True)
    plots.title('Regression Line: Estimate of True Line')

    sample.scatter('x', 'y', fit_line=True)
    #The following line plots the true line in green
    plots.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plots.title("Regression Line and True Line")

In [ ]:
draw_and_compare(2, -5, 10)
#Green is the true line

In [ ]:
draw_and_compare(2, -5, 10)

In [ ]:
draw_and_compare(2, -5, 100)

In [ ]:
draw_and_compare(2, -5, 100)

In [ ]:
draw_and_compare(2, -5, 1000)

SLIDE: Prediction Variability

## Prediction ##

In [ ]:
births = Table.read_table('baby.csv')

In [ ]:
births = Table.read_table('baby.csv')
births.show(3)

<h4>CDC Cutoffs/Definitions:<br>
    
Preterm Birth (baby born before 37 weeks of gestation)<br><br>
    
Postterm Birth (pregnancy extends to 42 weeks of gestation or more) </h4>

In [ ]:
# Preterm and postterm pregnancy cutoffs, according to the CDC
# Expressed in gestational days
37 * 7, 42 * 7

In [ ]:
births.scatter('Gestational Days', 'Birth Weight')

<h4>Discard the outliers.<br>
    
Keep only data where gestational days are between 225 and 325.</h4>

In [ ]:
births = births.where('Gestational Days', are.between(225, 325))

<h5>The following new scatter plot appears more zoomed-in, because we've discarded the outliers.</h5>

In [ ]:
births.scatter('Gestational Days', 'Birth Weight')

<h4>Now fit a regression line through the scatter plot.</h4>

In [ ]:
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)

<h5>Determine the correlation coefficient between Gestational Days and Birth Weight:</h5>

In [ ]:
correlation(births, 'Gestational Days', 'Birth Weight')

In [ ]:
def prediction_at(t, x, y, x_value):
    '''
    t - table
    x - label of x column
    y - label of y column
    x_value - the x value for which we want to predict y
    '''
    return slope(t, x, y) * x_value + intercept(t, x, y)

In [ ]:
prediction_at_300 = prediction_at(births, 'Gestational Days', 'Birth Weight', 300)
prediction_at_300

In [ ]:
x = 300
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)
plots.plot([x, x], [40, prediction_at_300], color='red', lw=2);

<h4>Now we look at the variability of the prediction.<br>
    
Each time we take a random sample, we obtain a different regression line and, therefore, a different prediction.</h5>

In [ ]:
# You don't need to understand the plotting code in this cell,
# but you should understand the figure that comes out.

plots.figure(figsize=(10, 11))
plots.subplot(3, 2, 1)
plots.scatter(births[1], births[0], s=10, color='darkblue')
plots.xlim([225, 325])
plots.title('Original sample')

#The following lines create 5 bootstrap samples of the Original Sample
for i in np.arange(1, 6, 1):
    plots.subplot(3,2,i+1)
    resampled = births.sample() #.sample() without argument defaults to full-size sample w/ replacement
    plots.scatter(resampled.column('Gestational Days'), resampled.column('Birth Weight'), s=10, color='tab:green')
    plots.xlim([225, 325])
    plots.title('Bootstrap sample '+str(i))
plots.tight_layout()

In [ ]:
for i in np.arange(4):
    resample = births.sample()
    predicted_y = prediction_at(resample, 'Gestational Days', 'Birth Weight', 300)
    print('Predicted y from bootstramp sample was', predicted_y)
    resample.scatter('Gestational Days', 'Birth Weight', fit_line=True)
    plots.scatter(300, predicted_y, color='gold', s=50, zorder=3);
    plots.plot([x, x], [40, predicted_y], color='red', lw=2);
    plots.plot([200, x], [predicted_y, predicted_y], color='red', lw=2);

<h4>Now resample 10 times, and plot the regression lines.<br>

Take ten different bootstraps.<br>

Determine the regression line for each bootstrap resample.<br>

Plot the regression lines on one chart for comparison.<br>

The dots are the predictions for the bootstraps.</h4>

In [ ]:
lines = Table(['slope','intercept', 'at 210', 'at 300', 'at 320'])

for i in range(10):
    resample = births.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 210 + b, a * 300 + b, a * 320 + b])

for i in np.arange(lines.num_rows):
    line = lines.row(i)
    plots.plot([210, 320], [line.item('at 210'), line.item('at 320')], lw=1)
    plots.scatter(300, line.item('at 300'), s=30, zorder=3)

<h4>Tabular values of the ten resamples.  Notice the predictions don't change that much (the "at 300" column):</h4>

In [ ]:
lines = Table(['slope','intercept', 'at 291', 'at 300', 'at 309'])

for i in range(10):
    resample = births.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 291 + b, a * 300 + b, a * 309 + b])
lines


<h4>Now let's zoom in to the neighborhood of 300 gestational days:</h4>

In [ ]:
for i in np.arange(lines.num_rows):
    line = lines.row(i)
    plots.plot([291, 309], [line.item('at 291'), line.item('at 309')], lw=1)
    plots.scatter(300, line.item('at 300'), s=30, zorder=3)

SLIDE: Confidence Interval for Prediction

## Confidence Interval for Prediction ##

In [ ]:
def bootstrap_prediction(t, x, y, new_x, repetitions=1000):
    """ 
    Makes a 95% confidence interval for the prediction at new_x, using
    linear regression on the data in t (column names x and y).
    Shows a histogram of the bootstrap samples and shows the interval
    in gold.
    """

    # Bootstrap the scatter, predict, collect
    predictions = make_array()
    for i in np.arange(repetitions):
        resample = t.sample()
        predicted_y = prediction_at(resample, x, y, new_x)
        predictions = np.append(predictions, predicted_y)

    # Find the ends of the approximate 95% prediction interval
    left = percentile(2.5, predictions)
    right = percentile(97.5, predictions)

    # Display results
    Table().with_column('Prediction', predictions).hist(bins=20)
    plots.xlabel('predictions at x='+str(new_x))
    plots.plot([left, right], [0, 0], color='gold', lw=8);
    print('Approximate 95%-confidence interval for height of true line:')
    print(np.round(left,2), np.round(right,2), 'width =', np.round(right - left,2), ')') 

In [ ]:
bootstrap_prediction(births, 'Gestational Days', 'Birth Weight', 300)

<h3> Predictions at Various Values of $x$</h3>

In [ ]:
x = 300
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)
plots.plot([x, x], [40, prediction_at_300], color='red', lw=2);

<h4>So far, we've only made predictions at $x=300$.<br>  
    
Let's see how the confidence intervals look for $x$ values to the extremes&mdash;far left and far right. </h4>

<h4>Let's how the prediction for $x=210$ (near the extreme left) varies:</h4>

In [ ]:
bootstrap_prediction(births, 'Gestational Days', 'Birth Weight', 210)

<h4>Let's how the prediction for $x=280$ (closer to the center) varies:</h4>

In [ ]:
bootstrap_prediction(births, 'Gestational Days', 'Birth Weight', 280)

<h4>Let's how the prediction for $x=325$ (near the extreme right) varies:</h4>

In [ ]:
bootstrap_prediction(births, 'Gestational Days', 'Birth Weight', 325)

<h4>Mean Gestational Days</h4>

In [ ]:
mean_gestational_days = np.mean(births.column('Gestational Days'))
np.round(mean_gestational_days,2)

SLIDE: Predictions at Different Values of $x$

<h3>Reason for Tighter Confidence Intervals Near the Center?</h3>

<h4>Every regression line goes through the center $(\mu_x,\mu_y)$ of the data, where $\mu_x$ and $\mu_y$ denote the Mean (Average) of $x$ and Mean (Average) of $y$, respectively.<br>

It's true that for each resample the center $(\mu_x,\mu_y)$ varies. But given a large enough bootstrap sample size, that variability is small. </h4> 



## Inference for the Slope ##

In [ ]:
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)

<h4>Regression Line Slope:</h4>

In [ ]:
slope(births, 'Gestational Days', 'Birth Weight')

<h4>Bootstrap 5,000 times:</h4>

In [ ]:
def bootstrap_slope(t, x, y, repetitions=5000):
    """ 
    Makes a 95% confidence interval for the slope of the prediction line
    for y, using linear regression on the data in t (column names x and y).
    Shows a histogram of the bootstrap samples and shows the interval
    in gold.
    """
    
    # Bootstrap the scatter, find the slope, collect
    slopes = make_array()
    for i in np.arange(repetitions):
        bootstrap_sample = t.sample()
        bootstrap_slope = slope(bootstrap_sample, x, y)
        slopes = np.append(slopes, bootstrap_slope)
    
    # Find the endpoints of the 95% confidence interval for the true slope
    left = percentile(2.5, slopes)
    right = percentile(97.5, slopes)
    
    # Slope of the regression line from the original sample
    observed_slope = slope(t, x, y)
    
    # Display results
    Table().with_column('Bootstrap Slopes', slopes).hist(bins=20)
    plots.plot(make_array(left, right), make_array(0, 0), color='yellow', lw=8);
    print('Slope of regression line:', np.round(observed_slope,2))
    print('Approximate 95%-confidence interval for the slope of the true line:')
    print(np.round(left,2), 'to', np.round(right,2))

In [ ]:
bootstrap_slope(births, 'Gestational Days', 'Birth Weight', 2500)

## Rain on the Regression Parade

In [ ]:
draw_and_compare(0, 10, 25)

**Null Hypothesis.** Slope of true line = 0.

**Alternative Hypothesis.** Slope of true line is not 0.

In [ ]:
slope(births, 'Maternal Age', 'Birth Weight')

In [ ]:
births.scatter('Maternal Age', 'Birth Weight', fit_line=True)

In [ ]:
bootstrap_slope(births, 'Maternal Age', 'Birth Weight', 2500)